In [1]:
import os
import shutil

In [2]:
train_dataset_path = '/Users/ap/Documents/MSAI/Q2/MSAI 437 DL /hw#2/beans/train'
valid_dataset_path = '/Users/ap/Documents/MSAI/Q2/MSAI 437 DL /hw#2/beans/valid'
subfolder_names = ['healthy', 'disease-ridden']
path_names = [train_dataset_path,valid_dataset_path]

#Creating subfolders in the dataset to represent the two classes.
for path_name in path_names:
    for subfolder_name in subfolder_names:
        subfolder_path = os.path.join(path_name, subfolder_name)
        os.makedirs(subfolder_path, exist_ok=True)


In [3]:
#Moving the data to their respective class folders
for path_name in path_names:
    for file_name in os.listdir(path_name):
        if (file_name.split('_')[0]=='healthy'):
            file_path = os.path.join(path_name, file_name)
            subfolder_path = os.path.join(path_name,'healthy')
            shutil.move(file_path, subfolder_path)
        else:
            file_path = os.path.join(path_name, file_name)
            subfolder_path = os.path.join(path_name,'disease-ridden')
            shutil.move(file_path, subfolder_path)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
val_dataset = datasets.ImageFolder(root=valid_dataset_path, transform=transform)


In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256,2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(-1, 64 * 28 * 28)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [15]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
criterion = nn.CrossEntropyLoss()

In [43]:
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Epoch {epoch+1}, Batch {i+1}: Loss {running_loss/100:.3f}")
            running_loss = 0.0

In [44]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on training set: {100 * correct / total:.2f}%")

Accuracy on training set: 99.03%


In [45]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total:.2f}%")

Accuracy on validation set: 90.98%
